In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle

In [203]:
with open('./data/video_statistic_features_one2.pickle','rb') as f2:
    temp_data = pickle.load(f2)

In [3]:
        with open('./data/video_raw_feature_sum_moving_average.pickle','rb') as f2:
            temp_data = pickle.load(f2)

In [4]:
temp_data

{'102844212428895431': array([0.        , 0.        , 0.        , ..., 0.13457849, 0.01878852,
        0.01444573]),
 '102844212429092040': array([0.        , 0.        , 0.        , ..., 0.018911  , 0.01511   ,
        0.00792267]),
 '102844212429419722': array([0.        , 0.        , 0.        , ..., 0.0243045 , 0.01570233,
        0.00831002]),
 '102844212429550795': array([0.        , 0.        , 0.        , ..., 0.16514719, 0.01869862,
        0.01338622]),
 '102844212429747404': array([0.        , 0.        , 0.        , ..., 0.0192366 , 0.01674523,
        0.01439629]),
 '102844212429944013': array([0.        , 0.        , 0.        , ..., 0.01838888, 0.01582532,
        0.01029176]),
 '102844212430075086': array([0.        , 0.        , 0.        , ..., 0.03960217, 0.03498364,
        0.02480106]),
 '102844212430402768': array([0.        , 0.        , 0.        , ..., 0.02681384, 0.02354126,
        0.02003448]),
 '102844212430599377': array([0.        , 0.        , 0.        

In [2]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    if epoch<20:
        lr = 0.01
    else:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

# LSTM model
output, feature를 한번에 뽑음

In [3]:
input_size=1
hidden_size=128
num_layers=3
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,3,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        out,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        ft = out[:,-1,:]
        out = self._lin(out[:,-1,:])
        return out,ft

# Dataset 준비
1. d_type : train, val, test 를 받음
2. 7초 window size shift 진행 --> *** 이부분 수정해야함.***
3. 하이라이트 구간 25%추출 

In [4]:
import torch.utils.data as data

class data_ds(data.Dataset):
    def __init__(self,d_type):
        self.gt_range =  1-0.25
        self.d_type=d_type
        
        #label load
        with open('label/label.pickle','rb') as f1:  
            self.gt=pickle.load(f1)
        
        #원본 데이터로드
        with open('./data/video_raw_feature_sum_moving_average.pickle','rb') as f2:
            temp_data = pickle.load(f2)
            if temp_data['102844412722519367'][0] is not list:
                for i in temp_data:
                    temp_data[i] = np.array(temp_data[i])[:,np.newaxis]
            self.data= temp_data
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
        if d_type == 'total':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654','102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630','102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']

        
        #sampling 대상이 될 데이터 저장 (25% 구간때문에 진행)
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.gt[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)

        #구간 구분하면서, 25% 만 추출 및 weightedsampling 에 저장
        begin_pos = 0 
        hl_frames = []
        for it, cur_pos in enumerate(pos_idx):
            if it+1 < len(pos_idx): 
                if((pos_idx[it+1] - cur_pos) > 1):#cur_pos와 cur_pos+1 간격이 1보다 크면, 즉 다른 구간이면
                    begin = int((it+1 - begin_pos) * self.gt_range) + begin_pos
                    hl_frames.extend( pos_idx[begin: it] ) #한구간의 하이라이트 25%만 사용하겠다.
                    begin_pos = it+1

        #sampling 비율 맞춰줌
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[hl_frames] = len(sampling) / float(len(hl_frames))
        self.WeightedSampling = sampling
        
        #sum : 각 경기당 데이터수 누적으로 저장 -->지금 어디 경기인지, 몇번째 프레임인지 확인할때 사용
        self.sum=np.insert(np.cumsum([len(self.gt[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return len(self.WeightedSampling)
    def __getitem__(self,index):
            global input_size
            #누적 데이터수 - index 로 프레임과 경기를 찾음
            vid=np.histogram(index,self.sum)
            vid = np.where(vid[0]>0)[0][0]
            vframe=index-self.sum[vid]
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            for idx in range(23): #7 : window size
                s_window=[]
                if vframe+idx<len(self.data[game_id]):
                    s_window=(self.data[game_id][vframe+idx])#vframe의 image
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*input_size # input_size
                window.append(s_window)
                


            #label 값 받기
            label=self.gt[str(game_id)][vframe]
            
            
            return np.array(window),label,str(game_id)
        
            
            



In [5]:
# 경기별 score 평균
def fmeasure2(frames,label):
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        print(precision,recall,accuracy)
        if precision==0 and recall == 0:
            print('error')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))


In [6]:
# 전체 데이터에 대해서 score
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

# Dataset load
train : undersampling 진행,  validation : 그냥 평등하게 sampling (sequential sampling)

In [7]:
class SampleSequentialSampler(sampler.Sampler):
    """Samples elements sequentially, always in the same order.
    Arguments:
        data_source (Dataset): dataset to sample from
        offset (int): offset between the samples
    """

    def __init__(self, data_source, offset=10):
        self.num_samples = len(data_source) 
        self.offset = offset

    def __iter__(self):
        return iter(np.arange(0, self.num_samples, self.offset ))

    def __len__(self):
        return len(np.arange(0, self.num_samples, self.offset ))


In [8]:
# train, validation dataset 준비
train=data_ds('train')
val=data_ds('val')

# dataloader with sampling
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=40000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
sampler2 =  SampleSequentialSampler(val, 30)
val_loader=torch.utils.data.DataLoader(val,batch_size=32,sampler= sampler2)

data load fin
data load fin


In [9]:
train[0]

(array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]), 0.0, '102844412722519367')

# model load 및 실험

- weight_dir/train_result 에 모든 결과가 저장됨.
- fmeasure (전체 데이터에 대한 score 로 weight를 저장..)

In [10]:
weight_dir='./video_raw_feature_sum_moving_average2/'

###### model load #####
model=LSTM().cuda()
criterion = nn.CrossEntropyLoss().cuda()

In [11]:
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

with open(weight_dir+'train_result','a') as f:


    best_losses=1000000
    for epoch in range(200):
        lr = adjust_learning_rate(optimizer, epoch)
        train_loss=0
        print(epoch)
        model.train()
        for i, (data,labels,game_id) in enumerate(train_loader):
            inputs = Variable(data).float().cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            output,_=model(inputs)

            loss=criterion(output,labels.long())
            train_loss+=loss
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.) #rnn 계열 gradient 장치
            loss.backward()
            optimizer.step()


        #validation
        model.eval()
        val_loss=0
        acc=0
        gt_sum=0
        val_result ={}
        with open(weight_dir+'train_result','a') as f:
            with torch.no_grad():
                for it, (data,labels,game_id) in enumerate(val_loader):
                    inputs = Variable(data).float().cuda()
                    labels = Variable(labels).cuda()
                    output,_=model(inputs)
                    loss=criterion(output,labels.long())
                    val_loss+=loss
                    TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
                    acc=acc+TP+TN
                    gt_sum+=len(output)
                val_acc=acc/gt_sum
                print("epoch {} train_loss : {} , val_loss : {}, val_acc : {}".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader),val_acc))
                f.write("epoch {} train_loss : {} , val_loss : {}, val_acc : {}\n".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader),val_acc))
                if best_losses>val_loss:
                    best_losses=val_loss
                    torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train_best"))#epoch 변화 확인위해
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"train_best"))#train best 덮어써짐

                    f.write("epoch {} saved\n".format(epoch))
                else:
                    torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))




0
epoch 0 train_loss : 0.6935967803001404 , val_loss : 0.6660415530204773, val_acc : 0.8261363636363637
1
epoch 1 train_loss : 0.6937519907951355 , val_loss : 0.710422933101654, val_acc : 0.17386363636363636
2
epoch 2 train_loss : 0.6937795877456665 , val_loss : 0.6839006543159485, val_acc : 0.8261363636363637
3
epoch 3 train_loss : 0.6937023401260376 , val_loss : 0.7093675136566162, val_acc : 0.17386363636363636
4
epoch 4 train_loss : 0.6936056613922119 , val_loss : 0.6907477974891663, val_acc : 0.8261363636363637
5
epoch 5 train_loss : 0.6935888528823853 , val_loss : 0.7176907062530518, val_acc : 0.17386363636363636
6
epoch 6 train_loss : 0.6937092542648315 , val_loss : 0.6690042018890381, val_acc : 0.8261363636363637
7
epoch 7 train_loss : 0.6935508847236633 , val_loss : 0.6514782309532166, val_acc : 0.8261363636363637
8
epoch 8 train_loss : 0.6934142112731934 , val_loss : 0.697493314743042, val_acc : 0.17386363636363636
9
epoch 9 train_loss : 0.6937239766120911 , val_loss : 0.70522

KeyboardInterrupt: 

# Test
- test set 로드
- best weight load
- weight_dir/train_result 에 결과 추가

In [33]:
#test


test=data_ds('test')#test dataset
test_loader=torch.utils.data.DataLoader(test,batch_size=32)#test dataloader
dataset=weight_dir+'7train' #best weight
checkpoint=torch.load(dataset,map_location='cuda:0')#model weight load
model.load_state_dict(checkpoint)
model.cuda()

#test exp
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (data,labels,g) in enumerate(test_loader):
        inputs = data.float().cuda()
        labels = labels.cuda()
        output,_=model(inputs)

        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,game_id in enumerate(g):
            if game_id not in result.keys():
                result[game_id]=pred[idx].tolist()
            else:
                result[game_id]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
2 0 27 3 tensor(2) tensor(5)
8 0 23 1 tensor(8) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 3 20 3 tensor(9) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 2 14 16 tensor(2) tensor(16)
7 0 15 10 tensor(7) tensor(17)
0 1 31 0 tensor(1) tensor(0)
0 0 32 0 tensor(0) tensor(0)
11 5 15 1 tensor(16) tensor(12)
0 7 25 0 tensor(7) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 15 17 0 tensor(15) tensor(0)
0 11 21 0 tensor(11) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 7 18 7 tensor(7) tensor(7)
13 0 14 5 tensor(13) tensor(18)
7 3 21 1 tensor(10) tensor(8)
0 12 20 0 tensor(12) tensor(0)
11 0 19 2 te

12 0 17 3 tensor(12) tensor(15)
7 0 23 2 tensor(7) tensor(9)
0 1 31 0 tensor(1) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 8 24 0 tensor(8) tensor(0)
3 1 28 0 tensor(4) tensor(3)
12 0 0 20 tensor(12) tensor(32)
16 0 12 4 tensor(16) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
24 0 3 5 tensor(24) tensor(29)
8 0 20 4 tensor(8) tensor(12)
0 10 22 0 tensor(10) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 9 23 0 tensor(9) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
11 0 11 10 tensor(11) tensor(21)
8 3 3 18 tensor(11) tensor(26)
0 0 32 0 tensor(0) tensor(0)
0 0 30 2 tensor(0) tensor(2)
8 0 12 12 tensor(8) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
3 0 27 2 tensor(3) tensor(5)
20 0 8 4 tensor(20) tensor(24)
0 0 25 7 tensor(0) tensor(7)
10 2 20 0 tensor(12) tensor(10)
0 0 32 0 tensor(

12 5 15 0 tensor(17) tensor(12)
16 4 8 4 tensor(20) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 2 30 0 tensor(2) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 0 23 0 tensor(9) tensor(9)
7 8 17 0 tensor(15) tensor(7)
0 1 29 2 tensor(1) tensor(2)
1 0 10 21 tensor(1) tensor(22)
26 0 6 0 tensor(26) tensor(26)
8 0 24 0 tensor(8) tensor(8)
13 0 9 10 tensor(13) tensor(23)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
14 0 13 5 tensor(14) tensor(19)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 2 23 2 tensor(7) tensor(7)
11 0 20 1 tensor(11) tensor(12)
0 0 30 2 tensor(0) tensor(2)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
17 4 9 2 tensor(21) tensor(19)
0 0 30 2 tensor(0) tensor(2)
8 1 12 11 tensor(9) tensor(19)
0 10 22 0 tensor(10) tensor(0)
0 20 12 0 tensor(2

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 0 25 1 tensor(6) tensor(7)
2 1 29 0 tensor(3) tensor(2)
0 0 32 0 tensor(0) tensor(0)
8 0 15 9 tensor(8) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 18 14 0 tensor(18) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 0 25 1 tensor(6) tensor(7)
0 1 31 0 tensor(1) tensor(0)
0 8 24 0 tensor(8) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 9 23 0 tensor(9) tensor(0)
0 0 32 0 tensor(0) tensor(0)
1 2 28 1 tensor(3) tensor(2)
6 1 25 0 tensor(7) tensor(6)
0 0 32 0 tensor(0) tensor(0)
0 12 20 0 tensor(12) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 11 21 0 tensor(11) tensor(0)
17 0 9 6 tensor(17) tensor(23)
10 2 19 1 tensor(12) tensor(11)
1 

In [34]:
with open('./label/label.pickle',"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

0.5352112676056338 0.6981627296587927 0.859577922077922
102844212431058132
precision : 0.5352112676056338, recall : 0.6981627296587927, f1 : 0.6059225512528473, accuracy : 0.859577922077922
0.5294117647058824 0.8537549407114624 0.8928905519176801
102844341902586509
precision : 0.5294117647058824, recall : 0.8537549407114624, f1 : 0.6535552193645991, accuracy : 0.8928905519176801
0.6738703339882122 0.8532338308457711 0.9017038007863696
102844401152267937
precision : 0.6738703339882122, recall : 0.8532338308457711, f1 : 0.7530186608122942, accuracy : 0.9017038007863696
0.7462962962962963 0.6106060606060606 0.896397580857218
102844212430927059
precision : 0.7462962962962963, recall : 0.6106060606060606, f1 : 0.6716666666666667, accuracy : 0.896397580857218
0.4825 0.6992753623188406 0.8589494163424124
102844412708953395
precision : 0.4825, recall : 0.6992753623188406, f1 : 0.5710059171597633, accuracy : 0.8589494163424124
0.6261904761904762 0.7088948787061995 0.8704789833822092
10284421242

# feature 추출 (마지막 LSTM feature)
- test, train, validation 모두 진행해야함!

In [24]:
save_dir = 'video_feature.pickle' #feature 어디에 저장할지

test=data_ds('total')
test_loader=torch.utils.data.DataLoader(test,batch_size=1)
dataset=weight_dir+'train_best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.cuda()
model.eval()

result={}
with torch.no_grad():
    for it, (data,labels,g) in enumerate(test_loader):
        inputs = data.float().cuda()
        labels = labels.cuda()
        _,output_f=model(inputs) #feature 만 추출

        if g[0] not in result.keys():
            print(g[0])
            result[g[0]]=[(output_f[0]).tolist()]
            
        else:
            result[g[0]]+=[(output_f[0]).tolist()]

with open(weight_dir+save_dir,'wb') as f:
    pickle.dump(result,f)       



data load fin
102844412722519367
102844212429550795
102844401151219358
102844401154430631
102844412717014335
102844401153971877
102844224148503678
102844412722847048
102844401152857762
102844412707380528
102844212431516886
102844283027925085
102844412716227901
102844412710001974
102844294670878922
102844294670551241
102844283023599703
102844412704496937
102844235751783874
102844401152071328
102844412709674293
102844401153447587
102844224148896895
102844235746868664
102979081290790284
102844283027531868
102844212431975640
102844401155937960
102844212429092040
102844341906649746
102844412706987311
102844412721339716
102844212430402768
102844341905011343
102844235753356742
102844235750997440
102844412709346612
102844412705217835
102844235752963525
102844412712164667
102844412705545516
102844341912220311
102844341907370644
102844235749424575
102844212429419722
102844294669568199
102844212431779031
102844294666422466
102844224146472059
102844212428895431
102844212429747404
10284423574870367

In [21]:
result['102844412722519367'][:10]

[[-0.35791340470314026,
  0.26698946952819824,
  -0.09512882679700851,
  -0.0047071343287825584,
  0.1654966175556183,
  0.14154905080795288,
  -0.3338702917098999,
  -0.12367624789476395,
  0.18952010571956635,
  -0.11053846031427383,
  -0.2618904411792755,
  -0.11754820495843887,
  0.17547045648097992,
  -0.03597959131002426,
  0.4411907196044922,
  0.014064689166843891,
  0.01666095107793808,
  0.1770620495080948,
  0.3251374661922455,
  0.003986166324466467,
  0.2823714315891266,
  -0.27818432450294495,
  0.37367361783981323,
  0.042778804898262024,
  0.28152137994766235,
  0.12793771922588348,
  0.2897695004940033,
  0.007131745107471943,
  0.19353102147579193,
  -0.2322344034910202,
  -0.13017696142196655,
  -0.04692450165748596,
  0.21807120740413666,
  0.21550297737121582,
  -0.08499440550804138,
  -0.3457574248313904,
  -0.3608167767524719,
  0.1919105052947998,
  -0.18918801844120026,
  0.13556288182735443,
  0.24390022456645966,
  0.23177573084831238,
  0.31884250044822693,
